### 1) Instalar dependências (opcional - rode no terminal ou em uma célula do notebook)
Em um notebook você pode executar (célula de bash):
```bash
pip install -U openai numpy scikit-learn python-dotenv langchain-google-genai requests
```
*Na prática, instale apenas as libs que vai usar no ambiente do Jupyter.*

**Nota:** Não é necessário instalar `google-generativeai` pois há conflito de dependências com `langchain-google-genai`. Usamos REST API direta para obter metadados completos.

In [1]:
import os
import csv
import numpy as np

from typing import List, Tuple, Optional
from openai import OpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pathlib import Path
from dotenv import load_dotenv

e:\01-projetos\11-work\11.34-engenharia-vetorial\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


### 2) Testar carregamento das variáveis de ambiente

In [2]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


# Checar chaves (rótulos simples)
print('OPENAI_API_KEY set? ->', bool(os.getenv('OPENAI_API_KEY')))
print('GOOGLE_API_KEY set? ->', bool(os.getenv('GOOGLE_API_KEY')))

🔎 .env carregado -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env
OPENAI_API_KEY set? -> True
GOOGLE_API_KEY set? -> True


### 3) Funções para gerar embeddings

- Vamos criar funções pequenas e claras para OpenAI e para Google Gemini (com fallback pra LangChain).
- Foque no fluxo: obter texto, chamar API, receber vetor.

In [3]:

# OpenAI embedding wrapper
def openai_embedding(text: str, model: str = 'text-embedding-3-small', return_usage: bool = False) -> Tuple[List[float], Optional[int]]:
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    resp = client.embeddings.create(input=text, model=model)
    emb = resp.data[0].embedding
    if return_usage:
        usage = getattr(resp, 'usage', None)
        total_tokens = getattr(usage, 'total_tokens', None) if usage is not None else None
        return emb, total_tokens
    return emb, None


#### Sobre a implementação REST da API Google

A função `google_embedding` usa duas abordagens diferentes:

**Quando `return_usage=True` (precisa de token count exato):**
- Faz chamada REST direta para `https://generativelanguage.googleapis.com/v1beta/models/{model}:embedContent`
- Retorna embedding + metadados incluindo `tokenCount` exato
- Se o token count não vier na resposta, faz uma segunda chamada para `:countTokens` endpoint
- Fallback para LangChain + estimativa se houver erro na API

**Quando `return_usage=False` (modo simples):**
- Usa `GoogleGenerativeAIEmbeddings` do LangChain (mais conveniente)
- Não faz chamadas extras para contar tokens

**Vantagens da abordagem REST:**
- ✅ Token count **exato** retornado pela API
- ✅ Sem conflitos de dependências
- ✅ Acesso a todos os metadados da resposta
- ✅ Fallback robusto em caso de erro

**Documentação oficial:**
- API Reference: https://ai.google.dev/api/rest/v1beta/models/embedContent
- Count Tokens: https://ai.google.dev/api/rest/v1beta/models/countTokens


In [4]:

# Google embedding wrapper (retorna embedding e tokens exatos via REST API)
def google_embedding(text: str, model: str = 'gemini-embedding-001', return_usage: bool = True) -> Tuple[List[float], Optional[int]]:        
    """
    Gera embeddings usando Google Gemini.
    
    Quando return_usage=True, usa REST API direta para obter token count exato.
    Quando return_usage=False, usa LangChain para simplicidade.
    """
    if return_usage:
        # Usar REST API direta para obter metadados completos incluindo token count
        import requests
        
        api_key = os.getenv('GOOGLE_API_KEY')
        if not api_key:
            raise ValueError('GOOGLE_API_KEY não encontrada nas variáveis de ambiente')
        
        url = f'https://generativelanguage.googleapis.com/v1beta/models/{model}:embedContent?key={api_key}'
        
        headers = {
            'Content-Type': 'application/json'
        }
        
        payload = {
            'model': f'models/{model}',
            'content': {
                'parts': [{'text': text}]
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            
            data = response.json()
            
            # Extrair embedding
            embedding = data.get('embedding', {}).get('values', [])
            
            # Extrair token count (pode estar em diferentes locais dependendo da versão da API)
            token_count = None
            
            # Tentar extrair de metadata
            if 'metadata' in data:
                token_count = data['metadata'].get('tokenCount')
            
            # Fallback: usar a API de count tokens se não vier na resposta
            if token_count is None:
                count_url = f'https://generativelanguage.googleapis.com/v1beta/models/{model}:countTokens?key={api_key}'
                count_payload = {
                    'contents': [{'parts': [{'text': text}]}]
                }
                count_response = requests.post(count_url, headers=headers, json=count_payload)
                if count_response.status_code == 200:
                    count_data = count_response.json()
                    token_count = count_data.get('totalTokens')
            
            return list(embedding), token_count
            
        except requests.exceptions.RequestException as e:
            print(f'Erro na chamada REST API do Google: {e}')
            # Fallback para LangChain com estimativa
            emb = GoogleGenerativeAIEmbeddings(model=model).embed_query(text)
            estimated_tokens = len(text) // 4
            return list(emb), estimated_tokens
    else:
        # Usar LangChain quando não precisar de usage (mais simples)
        emb = GoogleGenerativeAIEmbeddings(model=model).embed_query(text)
        return list(emb), None


### 4) Exemplo prático: gerar embeddings e comparar similaridade

Vamos gerar embeddings para 3 frases e calcular a similaridade (cosine).

In [5]:
# Exemplo e comparação: gerar embeddings e calcular similaridade

def cosine_sim(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

texts = [
    'O gato é um animal doméstico',
    'O gato é um felino de estimação',
    'A programação é importante para engenheiros de software'
]

emb1 = emb2 = emb3 = None
backend = None


### Como a similaridade é calculada (cosine similarity)

A similaridade coseno mede o quão "parecidos" são dois vetores de embeddings, comparando a **direção** deles no espaço vetorial (ignorando o tamanho/comprimento).

**Analogia intuitiva:**
Imagine dois vetores como setas no espaço. A similaridade coseno mede o ângulo entre essas setas:
- Se apontam na **mesma direção** (ângulo pequeno) → similaridade próxima de **1** (muito similares)
- Se apontam em **direções perpendiculares** (ângulo de 90°) → similaridade de **0** (sem relação)
- Se apontam em **direções opostas** (ângulo de 180°) → similaridade de **-1** (completamente opostos)

**Fórmula matemática:**

$$
\text{cosine\_sim}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \times \|v_2\|}
$$

Onde:
- $v_1 \cdot v_2$ é o **produto escalar** entre os vetores (soma dos produtos elemento a elemento)
- $\|v\|$ é a **norma L2** do vetor: $\sqrt{\sum x_i^2}$ (comprimento da "seta")
- O resultado está sempre entre **-1** e **+1**

**Por que é útil para embeddings?**

1. **Independente do tamanho**: Dividir pelo produto das normas "normaliza" os vetores, fazendo com que apenas a direção importe, não a magnitude
2. **Eficiente para comparar textos**: Textos semanticamente similares terão embeddings apontando em direções parecidas
3. **Normalização L2**: Muitos bancos vetoriais (como Qdrant) normalizam embeddings antes de armazenar, tornando a comparação ainda mais rápida (vira apenas produto escalar)

**Exemplo prático deste notebook:**
- **Sim 1-2 alta** (ex: 0.85): "gato doméstico" e "felino de estimação" têm significados próximos
- **Sim 1-3 baixa** (ex: 0.35): "gato doméstico" e "programação" são tópicos diferentes
- A diferença entre essas similaridades mostra que o modelo consegue distinguir bem os conceitos!

In [6]:

# Tentar OpenAI (pequeno) -> OpenAI (large) -> Google -> Erro amigável
openai_small_available = bool(os.getenv('OPENAI_API_KEY'))
openai_large_available = bool(os.getenv('OPENAI_API_KEY'))
google_available = bool(os.getenv('GOOGLE_API_KEY'))


In [7]:

# Função helper para rodar OpenAI e obter uso

def get_openai_embeddings(texts_list, model_name):
    embeddings = []
    total_tokens = 0
    for t in texts_list:
        emb, usage = openai_embedding(t, model=model_name, return_usage=True)
        embeddings.append(emb)
        if usage is not None:
            try:
                total_tokens += int(usage)
            except Exception:
                pass
    return embeddings, total_tokens if total_tokens else None


In [8]:
results = {}

# OpenAI small
if openai_small_available:
    try:
        emb_small, tokens_small = get_openai_embeddings(texts, 'text-embedding-3-small')
        results['openai_small'] = {'embeddings': emb_small, 'tokens': tokens_small, 'dim': len(emb_small[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI small:', e)


In [9]:
# OpenAI large
if openai_large_available:
    try:
        emb_large, tokens_large = get_openai_embeddings(texts, 'text-embedding-3-large')
        results['openai_large'] = {'embeddings': emb_large, 'tokens': tokens_large, 'dim': len(emb_large[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI large:', e)


In [10]:
# Google embeddings
if google_available:
    try:
        emb_google = [google_embedding(t, return_usage=True) for t in texts]
        results['google'] = {'embeddings': [e[0] for e in emb_google], 'tokens': [e[1] for e in emb_google], 'dim': len(emb_google[0][0])}
    except Exception as e:
        print('Falha ao gerar embeddings Google:', e)


In [11]:

# Mostrar resultados: dimensão, primeiros 5 valores e similaridades por backend
for k, v in results.items():
    print('\nBackend:', k)
    print('Dimensão:', v['dim'])
    
    # Exibir tokens (somar se for lista)
    tokens_value = v['tokens']
    if isinstance(tokens_value, list):
        total_tokens = sum(t for t in tokens_value if t is not None)
        print(f'Tokens (se disponível): {total_tokens} (total de {len(tokens_value)} chamadas)')
    else:
        print('Tokens (se disponível):', tokens_value)
    
    for i, e in enumerate(v['embeddings']):
        print(f' Embedding {i+1} primeiros 5:', e[:5])
    # calcular similaridade
    sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
    sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    print(f' Similaridade 1-2: {sim_12:.4f}')
    print(f' Similaridade 1-3: {sim_13:.4f}')



Backend: openai_small
Dimensão: 1536
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [0.001428517745807767, -0.004510881379246712, -0.04022705927491188, 0.04195206239819527, 0.015516397543251514]
 Embedding 2 primeiros 5: [-0.012964113615453243, 0.009385946206748486, -0.061422210186719894, 0.0561358705163002, 0.017261510714888573]
 Embedding 3 primeiros 5: [0.01946329139173031, 0.02798319421708584, -0.0038614224176853895, -0.009704718366265297, 0.048555903136730194]
 Similaridade 1-2: 0.8135
 Similaridade 1-3: 0.1421

Backend: openai_large
Dimensão: 3072
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [-0.021825000643730164, 0.03996645659208298, -0.0028245302382856607, 0.0035684334579855204, 0.010693789459764957]
 Embedding 2 primeiros 5: [-0.016434069722890854, 0.02808525785803795, -0.002878795610740781, -0.02375573106110096, 0.024798443540930748]
 Embedding 3 primeiros 5: [0.00042473155190236866, 0.0036773430183529854, -0.023540019989013672, 0.039040759205818176, -0.002575

### Como interpretar os resultados e o que estamos comparando

Nesta seção comparamos **três modelos** de embeddings: OpenAI (text-embedding-3-small), OpenAI (text-embedding-3-large) e Google (gemini-embedding-001). Para cada um comparamos:

- **Dimensão (dim)**: número de componentes no vetor de embedding. Modelos maiores normalmente representam mais informações, mas são mais caros e custam mais em armazenamento/consulta.
- **Tokens (contagem exata)**: número total de tokens usados nas chamadas, obtido diretamente das APIs:
  - OpenAI: via `response.usage.total_tokens`
  - Google: via REST API endpoints `:embedContent` ou `:countTokens`
  - Essencial para cálculo preciso de custos
- **Primeiros 5 valores do embedding**: rápido "check" para ver distribuição/escala dos vetores.
- **Similaridade 1-2 vs 1-3**: comparamos a similaridade entre a frase 1 e 2 (semântica próxima) e entre 1 e 3 (semântica diferente). O **esperado** é que 1-2 tenha similaridade maior que 1-3.

Interpretação:
- Se a similaridade 1-2 > 1-3, o modelo está capturando corretamente semântica local entre as frases; quanto maior a diferença, maior a separação semântica observada.
- Uma similaridade muito alta entre 1 e 3 sugere que o modelo não distingue bem os dois conceitos ou que as frases compartilham termos/estruturas que influenciam a representação.
- Use a dimensão e tokens para equilibrar custo vs qualidade: modelos com maior dimensão costumam retornar maior qualidade semântica, mas com custo e latência maiores.
- Compare os **tokens reais** (não estimados) entre modelos para entender diferenças na tokenização e no custo efetivo.

### Preços padrão 
Baseados em informações públicas (revisar e atualizar conforme o site oficial):

- OpenAI text-embedding-3-small: $0.02 por 1M tokens = $0.00002 por 1K tokens
- OpenAI text-embedding-3-large: $0.13 por 1M tokens = $0.00013 por 1K tokens
- Google gemini-embedding-001: $0.15 por 1M tokens = $0.00015 por 1K tokens

Referências:
- OpenAI Pricing: https://platform.openai.com/docs/models 
  - Selecione o embedding model na lista
  - Ver exemplo em: https://platform.openai.com/docs/models/text-embedding-3-small
- Google Gemini Pricing: https://ai.google.dev/gemini-api/docs/pricing#gemini-embedding

In [12]:
# Estimativa de custos (usa preços públicos quando possível)
from math import ceil

PRICING = {
    'openai_text-embedding-3-small': {'per_1k_tokens_usd': 0.00002},
    'openai_text-embedding-3-large': {'per_1k_tokens_usd': 0.00013},
    'google_gemini-embedding-001': {'per_1k_tokens_usd': 0.00015}
}


### Metodologia de estimativa de custos

Explicação de como calculamos o custo estimado por execução:

- Para modelos **OpenAI** (text-embedding-3-small e -large) usamos *tokens exatos* retornados no campo `usage.total_tokens` e aplicamos o custo por 1K tokens: `custo = (tokens / 1000) * price_per_1k_tokens`.
- Para modelo **Google** (gemini-embedding-001) usamos *tokens exatos* obtidos via REST API (endpoint `:embedContent` ou `:countTokens`) e aplicamos o custo por 1K tokens.
- `PRICING` é um dicionário editável que contém tarifas por 1k tokens. Esse dicionário pode ser atualizado manualmente com valores oficiais.
- Se houver falha ao obter tokens da API, usamos estimativa como fallback (1 token ≈ 4 caracteres).

**Fontes de token count:**
- OpenAI: `response.usage.total_tokens` (oficial)
- Google: REST API `embedContent` ou `countTokens` endpoint (oficial)

Limitações e recomendações:
- Os valores de **tokens são exatos** quando retornados pelas APIs oficiais.
- Os **preços** em `PRICING` são baseados em documentação pública e devem ser verificados periodicamente.
- Para custos de produção, valide com a documentação de preços atualizada (links: [OpenAI pricing](https://openai.com/pricing) e [Google AI Studio pricing](https://ai.google.dev/pricing)) e/ou use as APIs oficiais de faturamento, quando disponíveis.


In [13]:
# Função para calcular custo baseado em tokens exatos

def estimate_tokens_from_text(text: str) -> int:
    """
    Heurística para estimar tokens quando não disponíveis da API.
    Assume ~1.3 tokens por palavra (aproximação).
    """
    words = text.split()
    return max(1, int(ceil(len(words) * 1.3)))


def calculate_cost(model_key: str, tokens: int) -> float:
    """
    Calcula o custo baseado em tokens exatos e preços definidos em PRICING.
    
    Args:
        model_key: Chave do modelo no dicionário PRICING
        tokens: Número exato de tokens processados
    
    Returns:
        Custo em USD
    """
    pricing = PRICING.get(model_key)
    if pricing is None:
        raise KeyError(f'Modelo {model_key} sem preço definido no PRICING dict')

    if 'per_1k_tokens_usd' not in pricing:
        raise KeyError(f'Modelo {model_key} sem estrutura de preço válida')
    
    return (tokens / 1000.0) * pricing['per_1k_tokens_usd']


# Calcular custo para os resultados do notebook (results dict)
results_cost = {}

for k, v in results.items():
    # Mapear nome do resultado para chave do modelo
    model_key: str | None = None
    if k == 'openai_small':
        model_key = 'openai_text-embedding-3-small'
    elif k == 'openai_large':
        model_key = 'openai_text-embedding-3-large'
    elif k == 'google':
        model_key = 'google_gemini-embedding-001'
    
    # Validar que temos o model_key antes de continuar
    if model_key is None:
        print(f'⚠️  Modelo desconhecido: {k} - pulando cálculo de custo')
        results_cost[k] = None
        continue

    tokens = v.get('tokens')
    cost = None
    
    try:
        if tokens is not None:
            # Para Google, tokens é uma lista (um por chamada); somar todos
            if isinstance(tokens, list):
                total_tokens = sum(t for t in tokens if t is not None)
            else:
                total_tokens = tokens
            
            # Calcular custo com tokens reais
            if total_tokens > 0:
                cost = calculate_cost(model_key, total_tokens)
        else:
            # Fallback APENAS se tokens não disponível (não deveria acontecer)
            print(f'⚠️  Aviso: tokens não disponível para {k}, usando estimativa')
            total_tokens = sum(estimate_tokens_from_text(t) for t in texts)
            cost = calculate_cost(model_key, total_tokens)
            
    except Exception as e:
        print(f'❌ Erro ao calcular custo para {k}: {e}')
        cost = None
    
    results_cost[k] = cost

# Mostrar custos calculados
print('\n💰 Custos por modelo (esta execução):')
print('=' * 50)
for k, c in results_cost.items():
    if c is not None:
        print(f'{k:20} → ${c:.8f} USD')
    else:
        print(f'{k:20} → [erro no cálculo]')

print('\n📌 Observação: Valores baseados em tokens REAIS das APIs.')
print('   Revise PRICING periodicamente para manter preços atualizados.')


💰 Custos por modelo (esta execução):
openai_small         → $0.00000064 USD
openai_large         → $0.00000416 USD
google               → $0.00000360 USD

📌 Observação: Valores baseados em tokens REAIS das APIs.
   Revise PRICING periodicamente para manter preços atualizados.


In [15]:
# 6) Gerar relatório: CSV + gráficos + instruções para exportar slides/PDF


# tentativas de import para pandas/matplotlib; se faltar, gerar CSV somente
have_pandas = True
have_matplotlib = True
try:
    import pandas as pd
except Exception:
    have_pandas = False
    pd = None
try:
    import matplotlib.pyplot as plt
except Exception:
    have_matplotlib = False
    plt = None

# Criar pasta para salvar resultados
out_dir = Path('../../data') / 'embeddings'
out_dir.mkdir(parents=True, exist_ok=True)

# Montar linhas para CSV
rows = []
for k, v in results.items():
    try:
        sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
        sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    except Exception:
        sim_12 = None
        sim_13 = None
    
    # Normalizar tokens (somar se for lista)
    tokens_value = v.get('tokens')
    if isinstance(tokens_value, list):
        tokens_value = sum(t for t in tokens_value if t is not None)
    
    rows.append({
        'model': k,
        'dim': v.get('dim'),
        'tokens': tokens_value,
        'sim_12': sim_12,
        'sim_13': sim_13,
        'est_cost_usd': results_cost.get(k)
    })

csv_path = out_dir / 'comparative_results.csv'
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['model', 'dim', 'tokens', 'sim_12', 'sim_13', 'est_cost_usd'])
    writer.writeheader()
    for r in rows:
        writer.writerow(r)
print(f'Saved CSV report to {csv_path.resolve()}')

# If pandas available, show DataFrame
if have_pandas:
    df = pd.DataFrame(rows)
    
    # Configurar pandas para exibir números com mais casas decimais sem notação científica
    pd.set_option('display.float_format', lambda x: f'{x:.8f}')
    
    print('\nDataFrame summary:')
    display(df)
else:
    print('\nCSV content preview:')
    for r in rows:
        print(r)

# Tentar criar plots se matplotlib estiver disponível
if have_matplotlib and plt is not None:
    # Make arrays for plotting
    models = [r['model'] for r in rows]
    sim12 = [r['sim_12'] if r['sim_12'] is not None else 0 for r in rows]
    sim13 = [r['sim_13'] if r['sim_13'] is not None else 0 for r in rows]
    costs = [r['est_cost_usd'] if r['est_cost_usd'] is not None else 0 for r in rows]

    # Similarity bar chart
    plt.figure(figsize=(8, 4))
    x = range(len(models))
    plt.bar(x, sim12, width=0.4, label='Sim 1-2')
    plt.bar([i + 0.4 for i in x], sim13, width=0.4, label='Sim 1-3')
    plt.title('Similaridade (cosine) por modelo')
    plt.xticks([i + 0.2 for i in x], models)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plot1 = out_dir / 'similarity_comparison.png'
    plt.savefig(plot1)
    print(f'Saved plot to {plot1.resolve()}')
    plt.close()

    # Cost chart
    plt.figure(figsize=(6, 4))
    plt.bar(models, costs, color='orange')
    plt.title('Estimated cost per run (USD)')
    plt.ylabel('USD')
    plt.tight_layout()
    plot2 = out_dir / 'cost_comparison.png'
    plt.savefig(plot2)
    print(f'Saved plot to {plot2.resolve()}')
    plt.close()
else:
    print('\nMatplotlib or pandas not installed; plots skipped. To enable plots: pip install pandas matplotlib')

# Instruções para exportar o notebook em slides/PDF
print('\nInstruções para exportar o notebook como slides (Reveal.js) ou PDF:')
print('1) Para gerar slides HTML:')
print("   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'")
print('2) Para gerar PDF (pode exigir LaTeX):')
print('   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb')

# Também posso gerar esses arquivos automaticamente se preferir (recomendo usar o terminal).

Saved CSV report to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\comparative_results.csv

DataFrame summary:


,model,dim,tokens,sim_12,sim_13,est_cost_usd
0,openai_small,1536,32,0.81346465,0.14211046,0.00000064
1,openai_large,3072,32,0.84898137,0.23328540,0.00000416
2,google,3072,24,0.88463019,0.53127521,0.00000360


Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\similarity_comparison.png
Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\cost_comparison.png

Instruções para exportar o notebook como slides (Reveal.js) ou PDF:
1) Para gerar slides HTML:
   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'
2) Para gerar PDF (pode exigir LaTeX):
   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb
